In [2]:
from datasets import load_dataset
import pandas as pd
import os
import lightning.pytorch as pl
from torch.utils.data import DataLoader
import pickle
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq

/Users/pavlospoulos/miniconda3/envs/pavlosEnv2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset_name = 'Samsung/samsum'
dataset = load_dataset(dataset_name,  trust_remote_code=True)
print(dataset)

Generating validation split: 100%|██████████| 818/818 [00:00<00:00, 4090.45 examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})


In [6]:
dataset['train']['dialogue'][0]

"Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"

In [7]:
dataset['train']['summary'][0]

'Amanda baked cookies and will bring Jerry some tomorrow.'

In [8]:
class T5SummarizationDataModule(pl.LightningDataModule):
    def __init__(self, model_name, dataset_name, max_length, 
                 batch_size, train_range, val_range, test_range, seed_num):
        super().__init__()
        self.model_name = model_name
        self.dataset_name = dataset_name
        self.max_length = max_length
        self.batch_size = batch_size
        self.train_range = train_range
        self.val_range = val_range
        self.test_range = test_range
        self.seed_num = seed_num
        self.tokenizer = None
        self.data_collator = None
        self.train_dataset = 0
        self.val_dataset = 0
        self.test_dataset = 0
        self.cache_dir = f"./dataset_cache_{self.seed_num}"

    def prepare_data(self):
        # Downloading data, called only once on 1 GPU/TPU in distributed settings
        load_dataset(self.dataset_name,  trust_remote_code=True).shuffle(seed=self.seed_num)
        AutoTokenizer.from_pretrained(self.model_name)

    def setup(self, stage):
        # Setting up the data, called on every GPU/TPU in DDP
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.data_collator = DataCollatorForSeq2Seq(tokenizer=self.tokenizer, model=self.model_name)
        
        # Load and preprocess the dataset
        if stage == 'fit' or stage is None:
            self.train_dataset = self._get_or_process_dataset('train')
            self.val_dataset = self._get_or_process_dataset('val')
        if stage == 'test' or stage is None:
            self.test_dataset = self._get_or_process_dataset('test')
            
    def _get_or_process_dataset(self, split):
        cache_file = os.path.join(self.cache_dir, f"{split}_{self.seed_num}.pkl")
        
        if os.path.exists(cache_file):
            print(f"Loading cached {split} dataset...")
            with open(cache_file, 'rb') as f:
                return pickle.load(f)
        
        print(f"Processing {split} dataset...")
        dataset = load_dataset(self.dataset_name,  trust_remote_code=True).shuffle(seed=self.seed_num)
        
        if split == 'train':
            data = dataset['train'].select(range(min(self.train_range, len(dataset['train']))))
        elif split in ['val', 'test']:
            temp = dataset['test'].train_test_split(test_size=0.5, seed=self.seed_num, shuffle=True)
            if split == 'val':
                data = temp['train'].select(range(min(self.val_range, len(temp['train']))))
            else:
                data = temp['test'].select(range(min(self.test_range, len(temp['test']))))
        
        processed_dataset = self._preprocess_dataset(data)
        
        os.makedirs(self.cache_dir, exist_ok=True)
        with open(cache_file, 'wb') as f:
            pickle.dump(processed_dataset, f)
        
        return processed_dataset
    
    def _preprocess_dataset(self, dataset):
        return dataset.map(
            lambda x: self._preprocess_function(x),
            batched=True,
            remove_columns=dataset.column_names
        )
        
    def _preprocess_function(self, examples):
        prefix = "summarize: "
        inputs = [prefix + doc for doc in examples["dialogue"]]
        model_inputs = self.tokenizer(inputs, padding="max_length", 
                                      truncation=True, max_length=self.max_length)
        labels = self.tokenizer(text_target=examples["summary"], 
                                padding="max_length", truncation=True, max_length=self.max_length)
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, collate_fn=self.data_collator, shuffle=True, drop_last=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, collate_fn=self.data_collator, drop_last=True)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, collate_fn=self.data_collator, drop_last=True)


In [10]:
data_module = T5SummarizationDataModule(
    dataset_name='Samsung/samsum',
    model_name='t5-small',
    max_length=512,
    batch_size=32,
    train_range=20000,
    val_range=5000,
    test_range=5000,
    seed_num=42
)

data_module.prepare_data()
data_module.setup(stage='fit')

Processing train dataset...


Map: 100%|██████████| 14732/14732 [00:05<00:00, 2705.54 examples/s]


Processing val dataset...


Map: 100%|██████████| 409/409 [00:00<00:00, 2315.56 examples/s]


In [11]:
dataloader = next(iter(data_module.train_dataloader()))


In [12]:
data_module.tokenizer.batch_decode(dataloader['input_ids'][0])

['summarize',
 ':',
 'A',
 'ly',
 's',
 's',
 'a',
 ':',
 'have',
 'you',
 'seen',
 'this',
 'fight',
 'on',
 '',
 'f',
 'b',
 '??',
 'A',
 'ly',
 's',
 's',
 'a',
 ':',
 'on',
 'the',
 'group',
 'chat',
 'dedicated',
 'to',
 'planning',
 'Alice',
 "'",
 's',
 'bachelor',
 'ette',
 'party',
 'Ari',
 'anna',
 ':',
 '',
 'o',
 'h',
 ',',
 '',
 'f',
 'uck',
 'Ari',
 'anna',
 ':',
 'give',
 'ma',
 '',
 'a',
 'sec',
 ',',
 '',
 'i',
 "'",
 'll',
 'catch',
 'up',
 'on',
 'everything',
 'and',
 '',
 'i',
 "'",
 'll',
 'be',
 'back',
 'A',
 'ly',
 's',
 's',
 'a',
 ':',
 '',
 'ok',
 '',
 ':',
 ')',
 'Ari',
 'anna',
 ':',
 'what',
 '',
 'a',
 '',
 's',
 'hit',
 'storm',
 'Ari',
 'anna',
 ':',
 'have',
 'you',
 'noticed',
 ',',
 'that',
 'this',
 'dumb',
 'a',
 's',
 's',
 'Julian',
 'a',
 'added',
 'Alice',
 'to',
 'the',
 'group',
 'chat',
 '?',
 'Ari',
 'anna',
 ':',
 'talk',
 'about',
 'surprise',
 'party',
 'A',
 'ly',
 's',
 's',
 'a',
 ':',
 '',
 'o',
 'm',
 'g',
 'no',
 '!',
 'A',
 'ly',

In [13]:
data_module.tokenizer.batch_decode(dataloader['labels'][0])

['There',
 'was',
 '',
 'a',
 'fight',
 'on',
 '',
 'a',
 '',
 'f',
 'b',
 'group',
 'chat',
 'between',
 'girls',
 ',',
 'concerning',
 'details',
 'for',
 'Alice',
 "'",
 's',
 'surprise',
 'party',
 '.',
 'Three',
 'girls',
 ',',
 '',
 'whose',
 'idea',
 'of',
 'organizing',
 'male',
 'strip',
 'per',
 'was',
 'rejected',
 ',',
 'started',
 'calling',
 'others',
 'names',
 '.',
 '</s>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>